In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [2]:
train = pd.read_csv('normalized2_train.csv', index_col='id', parse_dates=['fecha'])
test = pd.read_csv('normalized2_test.csv', index_col='id', parse_dates=['fecha'])

In [3]:
# Casteamos todo a int, esto es porque XgBoost dio problemas usando el set de datos con los datos heredados.
const = ['antiguedad', 'banos', 'garages', 'metroscubiertos', 'metrostotales', 'metros']
every_column_except_y= [col for col in train.columns if col not in ['preciomt2', 'precio','id', 'logpreciomt2', 'fecha', 'descripcion', 'titulo', 'logprecio']]
columns = [col for col in train[every_column_except_y].columns if col not in const]
# columns
train.loc[:, columns] = train[columns].fillna(0)
for column in columns:
    train[column] = train[column].astype('int32')

In [4]:
xgb = XGBRegressor(objective ='reg:squarederror',\
                   colsample_bytree = 0.8,\
                   learning_rate = 0.1,\
                   max_depth = 10,\
                   reg_alpha = 1.2,\
                   n_estimators = 1000,\
                   reg_lambda = 1.4,\
                   subsample=0.8, seed=30)

In [5]:
X, y = train.loc[:,every_column_except_y], train['logprecio']

In [6]:
xgb.fit(train[every_column_except_y], train['logpreciomt2'])

/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=1.2, reg_lambda=1.4, scale_pos_weight=1,
             seed=30, silent=None, subsample=0.8, verbosity=1)

In [7]:
pred = xgb.predict(test[every_column_except_y])

In [8]:
final = pd.DataFrame(test.index)
final.loc[:, 'target'] = np.array(test.metros) * np.expm1(pred)
pred_final = final.set_index('id')

In [9]:
pred

array([14.115986, 11.898907, 12.54506 , ..., 12.299816, 11.771861,
       12.604658], dtype=float32)

In [10]:
pred_final

,target
id,
4941,7.707450e+06
51775,1.241419e+06
115253,2.552291e+06
299321,1.422726e+06
173570,5.318680e+05
...,...
75094,3.527576e+06
171847,7.737907e+05
138313,1.018040e+06


In [11]:
rf = RandomForestRegressor(n_estimators=1000, min_samples_split=5, min_samples_leaf=2,\
                           max_features='sqrt', max_depth=60, bootstrap= False)

# aca deberia armar las variables X_train e y_train, llenando los nans de df_train

train.fillna(0, inplace=True)

rf.fit(train[every_column_except_y], train['logpreciomt2'])



RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=60,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [15]:
preds = rf.predict(test[every_column_except_y].fillna(0))

In [16]:
#preds = rf.predict(test[every_column_except_y])
final = pd.DataFrame(test.index)
final.loc[:, 'target'] = np.array(test.metros) * np.expm1(pred)
pred_final_rf = final.set_index('id')

In [17]:
pred_total = (0.6*pred_final+0.4*pred_final_rf)

In [20]:
res = pd.DataFrame(pred_total, index=test.index, columns=['target'])
display(res.head())
res.to_csv("submission-rf-xgb-text.csv", header=True)

,target
id,
4941,7.707450e+06
51775,1.241419e+06
115253,2.552291e+06
299321,1.422726e+06
173570,5.318680e+05


# SCORE: 542